<a href="https://colab.research.google.com/github/toainn92/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-20 04:35:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-20 04:35:11 (6.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
#create vine_table Dataframe

vine_df= df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']).drop_duplicates()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15RUMO9UBZ4RK|          4|            0|          0|   N|                Y|
| R82KK9VVQPR60|          2|            0|          0|   N|                Y|
|R3BRM9YAPUUFOI|          5|            1|          1|   N|                Y|
|R18M6JWDUB4QZX|          5|            0|          0|   N|                Y|
|R36015CE2Q0MEH|          5|            0|          0|   N|                Y|
|R1NRNWIQBRTRRY|          5|            0|          0|   N|                Y|
| RE1CN1JMPQCSJ|          5|            0|          0|   N|                Y|
|R3VCRSXMSJEXYI|          4|            0|          0|   N|                Y|
| RZ5DTD6Q7URBK|          5|            0|          0|   N|                Y|
|R35DFS8G6JRE12|          4|            0|          1|   N|     

In [10]:
# Filter Dataframe for total_votes greater than 20
vote_20_df=vine_df.filter(vine_df.total_votes>=20)
vote_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PD5YTU6HJMQA|          1|            0|         24|   N|                N|
|R104OBYLJ6UBFB|          1|           14|         22|   N|                Y|
|R2OXUUOSXHYQ13|          1|            4|         31|   N|                Y|
| R4NE1F0DEFVNK|          5|           30|         89|   N|                N|
|R24CALEW9UCXO6|          1|           21|         22|   N|                N|
|R2T0SEKG379S3O|          1|            3|         25|   N|                Y|
|R3HUPV7MRE5LLT|          1|           41|         46|   N|                N|
|R3Q53V8VUGECR7|          3|           46|         92|   N|                Y|
|R1Y4P07NRA7ZSL|          1|           26|        349|   N|                Y|
|R3DPAQTP4A27OG|          1|           64|         74|   N|     

In [14]:
#Filter Dataframe to retrieve all rows where hte number of helpful_votes divided by total_votes is equal or greater than 50%
helpful_50_df=vote_20_df.filter((vote_20_df.helpful_votes/vote_20_df.total_votes) >= 0.5)
helpful_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R104OBYLJ6UBFB|          1|           14|         22|   N|                Y|
|R24CALEW9UCXO6|          1|           21|         22|   N|                N|
|R3HUPV7MRE5LLT|          1|           41|         46|   N|                N|
|R3Q53V8VUGECR7|          3|           46|         92|   N|                Y|
|R3DPAQTP4A27OG|          1|           64|         74|   N|                Y|
| RHQN2K54NUBG1|          3|           32|         37|   N|                N|
| RUNSXNUAESPZ9|          5|           25|         26|   N|                Y|
|R1DFY65UMMEMNJ|          5|           30|         30|   N|                Y|
| R9JFSFCZBP05E|          4|           14|         20|   N|                N|
|R3DEQDS41493O8|          5|           83|        146|   N|     

In [17]:
#Create dataframe that retrieves all rows where a review was written as part of the Vine Program (paid)
vine_paid_df=helpful_50_df.filter(helpful_50_df.vine =='Y')
vine_paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [18]:
#Create dataframe that retrieves all rows where a review was written as not part of the Vine Program (unpaid)
vine_unpaid_df=helpful_50_df.filter(helpful_50_df.vine =='N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R104OBYLJ6UBFB|          1|           14|         22|   N|                Y|
|R24CALEW9UCXO6|          1|           21|         22|   N|                N|
|R3HUPV7MRE5LLT|          1|           41|         46|   N|                N|
|R3Q53V8VUGECR7|          3|           46|         92|   N|                Y|
|R3DPAQTP4A27OG|          1|           64|         74|   N|                Y|
| RHQN2K54NUBG1|          3|           32|         37|   N|                N|
| RUNSXNUAESPZ9|          5|           25|         26|   N|                Y|
|R1DFY65UMMEMNJ|          5|           30|         30|   N|                Y|
| R9JFSFCZBP05E|          4|           14|         20|   N|                N|
|R3DEQDS41493O8|          5|           83|        146|   N|     

In [19]:
#Vine Statistics

#Vine Total number of review
vine_total=helpful_50_df.count()
vine_total

1685

In [21]:
#The Number of 5-star review
five_star_review= helpful_50_df.filter(helpful_50_df.star_rating ==5).count()
five_star_review

631

In [24]:
#Total number of reviews with Vine
vine_reviews= vine_paid_df.count()
vine_reviews

0

In [25]:
#The number of 5-star review with Vine review
five_star_vine= vine_paid_df.filter(vine_paid_df.star_rating==5).count()
five_star_vine

0

In [26]:
#Total number of reviews with no Vine
no_vine_reviews=vine_unpaid_df.count()
no_vine_reviews

1685

In [27]:
#The number of 5-star review with no Vine review
five_star_no_vine=vine_unpaid_df.filter(vine_unpaid_df.star_rating==5).count()
five_star_no_vine

631

In [31]:
#Percentage of 5 star vine review
vine_5star_percentage= (five_star_vine/five_star_review)*100
vine_5star_percentage

0.0

In [41]:
#Percentage of 5 star no vine review
No_vine_5star_percentage=(five_star_no_vine/no_vine_reviews)*100
No_vine_5star_percentage

37.448071216617215

In [42]:
#Summary Statistics
print("Total Reviews=", vine_total)

print("Total Vine Review= ", vine_reviews)
print("Total Vine 5 Star Review=", five_star_vine)
print("Percentage of 5 star Vine Review= ", vine_5star_percentage)

print("Total Non Vine Review= ", no_vine_reviews)
print("Total Non Vine 5 Star Review= ", five_star_no_vine)
print("Percentage of 5 Star No-Vine Review= ", No_vine_5star_percentage)

Total Reviews= 1685
Total Vine Review=  0
Total Vine 5 Star Review= 0
Percentage of 5 star Vine Review=  0.0
Total Non Vine Review=  1685
Total Non Vine 5 Star Review=  631
Percentage of 5 Star No-Vine Review=  37.448071216617215
